In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.stats.stats import pearsonr
import pickle
import statistics

In [2]:
# load dataset
padel_desc = pd.read_csv(r'data/padel_desc.csv')
pIC50_target = pd.read_excel(r'data/pIC50_target.xlsx')

In [3]:
#kolom yang dihapus
padel_desc = padel_desc.drop(["Name"], axis=1)
pIC50_target = pIC50_target.drop(['No.'], axis=1)

padel_desc.shape

(134, 1875)

In [4]:
dataset = pd.concat([padel_desc, pIC50_target], axis=1)
dataset.head()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
0,0,-1.0627,1.129331,79.6556,45.508860,6,6,42,22,20,...,0.230608,0.561977,0.489222,0.523529,13.444158,47.047030,106.927129,0.459836,1.574728,5.842
1,0,-0.0693,0.004802,113.5328,61.298032,0,0,54,30,24,...,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973,6.292
2,0,-1.2672,1.605796,89.2215,51.164446,6,6,47,25,22,...,0.093133,0.554587,0.403332,0.402626,21.435407,63.448720,134.077300,0.764904,1.360546,5.469
3,0,-1.2656,1.601743,119.6603,64.662032,12,12,57,33,24,...,0.123379,0.544054,0.403809,0.370641,28.756514,130.576181,305.286387,0.723444,1.318503,6.310
4,0,-1.6474,2.713927,110.0384,59.971239,12,12,53,30,23,...,0.141888,0.581195,0.397869,0.322742,24.574375,101.480573,223.519513,0.700411,1.301806,6.387


In [5]:
# drop kolom yang ada duplikat
dataset = dataset.loc[:,~dataset.columns.duplicated()]
dataset.shape

(134, 1876)

In [6]:
dataset.corr()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
nAcid,1.000000,-0.114236,0.006209,0.088071,0.013621,0.000203,-0.013937,0.022004,0.160001,-0.156621,...,-0.049143,0.107451,0.081702,-0.213401,0.119141,0.052404,0.032889,0.096530,-0.013977,-0.213024
ALogP,-0.114236,1.000000,-0.857360,0.254768,0.209999,0.076592,0.072968,0.195940,0.257864,0.065564,...,0.044969,-0.089465,-0.111621,0.046016,0.231345,0.253177,0.228870,-0.019906,-0.080229,-0.006022
ALogp2,0.006209,-0.857360,1.000000,-0.163429,-0.126159,-0.063020,-0.063817,-0.117394,-0.179896,-0.007494,...,-0.075377,-0.050359,0.060835,0.061681,-0.159146,-0.195997,-0.172608,0.040654,0.042470,0.017536
AMR,0.088071,0.254768,-0.163429,1.000000,0.968285,0.455602,0.469023,0.914469,0.940762,0.634745,...,0.235781,-0.454176,-0.190383,-0.192919,0.625950,0.844764,0.815564,-0.079789,-0.414871,-0.180608
apol,0.013621,0.209999,-0.126159,0.968285,1.000000,0.398751,0.415852,0.973154,0.922022,0.774478,...,0.222736,-0.402452,-0.182370,-0.153073,0.633427,0.864848,0.842900,-0.081063,-0.366129,-0.022001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
As,0.052404,0.253177,-0.195997,0.844764,0.864848,0.251537,0.262301,0.855214,0.854179,0.625681,...,0.114441,-0.250555,-0.185884,-0.095836,0.759033,1.000000,0.975356,0.032995,-0.268761,-0.079064
Vs,0.032889,0.228870,-0.172608,0.815564,0.842900,0.247320,0.256754,0.838185,0.837691,0.612570,...,0.190884,-0.243072,-0.148348,-0.022844,0.668465,0.975356,1.000000,-0.084704,-0.206891,-0.054257
Ks,0.096530,-0.019906,0.040654,-0.079789,-0.081063,-0.186128,-0.190967,-0.028471,-0.103837,0.073525,...,-0.947063,0.187022,-0.268871,-0.321080,0.616984,0.032995,-0.084704,1.000000,-0.228918,-0.199511
Ds,-0.013977,-0.080229,0.042470,-0.414871,-0.366129,-0.337996,-0.338530,-0.334676,-0.301915,-0.285341,...,0.139306,0.640900,0.709026,0.599791,-0.371771,-0.268761,-0.206891,-0.228918,1.000000,0.178652


In [7]:
# drop kolom yang ada 0
dataset = dataset[(dataset.sum(axis=1) != 0)]
dataset.shape

(134, 1876)

In [8]:
# drop kolom yang ada nan
dataset.dropna(axis=1, inplace=True)
dataset.shape

(134, 1876)

In [9]:
dataset.isna().values.any()

False

In [10]:
train, test = train_test_split(dataset, test_size = 0.2, random_state = 10)

train.shape, test.shape
# test

((107, 1876), (27, 1876))

In [20]:
pickle.dump(train, open('data/train_bams.p','wb'))
pickle.dump(test, open('data/test_bams.p','wb'))

In [11]:
corr = train.corr()
corr = corr.iloc[:-1,[-1]]
corr.sort_values("pIC50", ascending= False, inplace = True)
corr = corr.iloc[:100,:]
corr.to_csv (r'data/data_corr.csv', index = None, header=True)

In [15]:
# -----------REMOVE FEATURE WITH LOW CORRELATION TO Y RESPONSE---------------------
label_ = train.columns.values
pic50 = train.iloc[:,-1]
corr_y = [pearsonr(train.iloc[:,i],pic50) for i in range(train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(train.shape[1])]
corr_lim = 0.20
hi_corr = []
for i in range(len(corr_y)):
    if corr_y[i] > corr_lim:
        hi_corr.append(i)

# label_idx = label_[hi_corr]
# train = train.loc[:,label_idx]
# print("descriptor number after removing low correlation with target: {}".format(train.shape[1]))

C:\Users\Bams\anaconda3\lib\site-packages\scipy\stats\stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [16]:
# -----------REMOVE FEATURE WITH HIGH CORRELATION TO OTHER FEATURE---------------------
# re-calculate correlation with pic50
label_ = train.columns.values
corr_y = [pearsonr(train.iloc[:,i],pic50) for i in range(train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(train.shape[1])]
desc_num = train.shape[1]
# calculate correlation for each descriptor
corr_matrix = np.corrcoef(train.T)
corr_lim = 0.80
low_corr = np.arange(desc_num).tolist()
tmp = np.arange(desc_num).tolist()
for i in np.arange(desc_num):
    tmp.remove(i)
    for j in tmp:
        corr_ = np.abs(corr_matrix[i,j])
        if corr_ >= corr_lim:
            if corr_y[i] > corr_y[j]:
                if j in low_corr:
                    low_corr.remove(j)
            else:
                if i in low_corr:
                    low_corr.remove(i)
label_idx = label_[low_corr]
train = train.loc[:,label_idx]
print("descriptor number after removing high correlation descriptor: {}".format(train.shape[1]))

C:\Users\Bams\anaconda3\lib\site-packages\numpy\lib\function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Bams\anaconda3\lib\site-packages\numpy\lib\function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


descriptor number after removing high correlation descriptor: 578


In [17]:
train.shape, test.shape

((107, 578), (27, 1876))

In [ ]:
#print datatest dan datatrain
test.to_csv('data/new_test.csv', encoding='utf-8')
train.to_csv('data/new_train.csv', encoding='utf-8')

In [ ]:
label_ = train.columns.values
pickle.dump(label_, open('data/new_corr_Label.p','wb'))

In [ ]:
# dataset.describe()